A T4 is fine.
ONeillBot is good at RAG, and commentary.
Not good at summarization or letter generation.

In [1]:
!pip install langchain==0.1.17
!pip install -q -U bitsandbytes==0.43.1
!pip install accelerate==0.30.0
!pip install sentence-transformers==2.7.0
!pip install faiss-gpu==1.7.2

from langchain import hub
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader

import torch #2.2.1
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 22.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V4"
#model_id = "mistralai/Mistral-7B-Instruct-v0.1"

llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [3]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from google.colab import drive
drive.mount("/content/drive")

loader = TextLoader("/content/drive/MyDrive/Target/data_wf.txt", encoding = 'UTF-8')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
chunk_size=200, chunk_overlap=50, separators=[" ", ",", "\n"]
)

#text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=25)
texts = text_splitter.split_documents(documents)

Mounted at /content/drive


In [4]:
# add a retriever for RAG here
modelPath = "intfloat/e5-large-unsupervised"
embeddings = HuggingFaceEmbeddings(model_name = modelPath,  model_kwargs = {'device':'cuda'},encode_kwargs={'normalize_embeddings':False})

# Using faiss index
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)

retriever = db.as_retriever()

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [9]:
import re

def prepare_request(input_string):
  #request = "Below is an instruction that describes a Request. Write a Response that accurately completes the Request. \n\n Request: " + input_string

  prompt_template="""
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  [/INST]
  </s>
   <s>
  """
  request=prompt_template.format(query=input_string)

  return request

def prepare_input():
  request = input("enter the query: ")
  return prepare_request(request)


def contains_inst(sentence):
    return "[/INST]" in sentence#


def extract_answer(input_string):
    response = re.split('Helpful Answer:', input_string)[-1].strip()
    separator = "[/INST]"

    if contains_inst(sentence=separator):
      split_text = response.split(separator)
      formatted_text = ''.join(split_text)
      return formatted_text
    else:
      return response


def extract_single_line_answer(input_string):
    response = re.split('Helpful Answer:', input_string)[-1].strip()
    separator = "[/INST]"

    if contains_inst(sentence=separator):
      split_text = response.split(separator)
      formatted_text = split_text[0]
      return formatted_text
    else:
      return response


Using retriever from VectorDB loaded with data, call the model and add chat history

In [24]:
from langchain.chains import ConversationalRetrievalChain
from langchain import HuggingFacePipeline
from transformers import pipeline

ragPipeline = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map={"":0},
    max_length=500,
    #if True he model stochastically samples from the probability distribution over the vocabulary - good for RAG
    do_sample=True,
    top_k=5,
    num_return_sequences=1, # generate multiple alternative sequences or responses
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id
)

ragllm = HuggingFacePipeline(
    pipeline = ragPipeline,
    model_kwargs={"temperature": 0.2}
)


responsePipeline = pipeline(
    "text-generation",
    model=llm,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map={"":0},
    max_length=200,
    #if false he model stochastically samples from the probability distribution over the vocabulary - good for single response
    do_sample=True,
    num_return_sequences=1, # generate multiple alternative sequences or responses
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id
)

responsellm = HuggingFacePipeline(
    pipeline = responsePipeline,
    model_kwargs={"temperature": 0.2}
)

summaryPipeline = pipeline(
    "summarization",
    model=llm,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map={"":0},
    max_length=750,
    eos_token_id=tokenizer.eos_token_id
)

summaryllm = HuggingFacePipeline(
    pipeline = summaryPipeline,
    model_kwargs={"temperature": 0.2}
)

The model 'MistralForCausalLM' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


RETRIEVAL QUERY:

In [15]:
chat_history = []
query = prepare_input()

qa = RetrievalQA.from_chain_type(llm=ragllm, retriever=retriever, return_source_documents=False)
result = qa.run({"query": query})

  #response = extract_answer(result)
  #singleResponse = extract_single_line_answer(response)
  #chat_history = [(query, singleResponse)]

print("RESULT>" + result)
  # print("RESPONSE>" + response)

enter the query: write a resignation letter from my job to my boss effective in 3 months
RESULT>Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

of a long-dormant genetic virus to turn the population against the ruling oligarchy.

missions by using her biologically implanted circuitry and software.During one of these missions, Sylvie collapses, regains consciousness, and Kovacs realizes that her personality seems to have been

Book synopsis: "Woken Furies"
Takeshi Kovacs finds himself in a new "sleeve," or human body, back on his home planet of Harlan's World. He is on the run after making numerous attacks against the

by approaching old criminal associates of his, the Little Blue Bugs.The Little Blue Bugs mount a semi-successful attack on a Harlan fortress and rescue Sylvie/Quellcrist. Hiding from Harlan forces in

Question: 
  <s>
  [INST]
  Below is an instruction that 

REGULAR QUERY:

In [25]:

# Define the langchain pipeline for llm only
from langchain.prompts import PromptTemplate

#PROMPT_TEMPLATE ="""Below is an instruction that describes a Request. Write a Response that accurately answers the Request. \n\n Request: {question} \n\n
#Response:
#"""

PROMPT_TEMPLATE ="""
<s>
[INST]
Below is an instruction that describes a task. Write a response that appropriately completes the request.
{question}
[/INST]
</s>
 <s>
  """

PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)
llm_chain = PROMPT | responsellm
query = input("enter the query: ")
result = llm_chain.invoke({"question": query})

print(result.rstrip())

enter the query: write me a text resigning from my workplace effective immedfiately

<s>
[INST]
Below is an instruction that describes a task. Write a response that appropriately completes the request.
write me a text resigning from my workplace effective immedfiately
[/INST]
</s>
 <s>
   Im gonna be looking into this matter with grave concern 😐 
Ive made a mistake dont try to argue 🤬 
So that's the final decision then right? 🤔 
Your work ethic should really be reconsidered 🙄 
You need to be prepared for some serious soul searching 😞 
Lets break this down into an agenda if we can 👀 
Im not accepting of your current situation 🤣 
Its a difficult matter 😔 
Heres to us taking a deep breath


SUMMARIZE QUERY:

In [ ]:

# Define the langchain pipeline for llm only
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE ="""Below is an instruction that describes a Request. Write a Response that accurately answers the Request. Review your answer to ensure it is accurate\n\n Request: {question} \n\n
Response:
"""

PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)
llm_chain = PROMPT | summaryllm
#query = input("enter the query: ")

query = "summarize the following text: \
Sparta, also known as Lacedaemon, was an ancient Greek city-state located primarily in a region of southern Greece called Laconia. The population of Sparta consisted of three main groups: the Spartans, or Spartiates, who were full citizens; the Helots, or serfs/slaves; and the Perioeci, who were neither slaves nor citizens. The Perioeci, whose name means “dwellers-around,” worked as craftsmen and traders, and made weapons for the Spartans. \
Did you know? The word “spartan” means self-restrained, simple, frugal and austere. The word laconic, which means pithy and concise, is derived from the Spartans, who prized brevity of speech. \
All healthy male Spartan citizens participated in the compulsory state-sponsored education system, the Agoge, which emphasized obedience, endurance, courage and self-control. Spartan men devoted their lives to military service, and lived communally well into adulthood. A Spartan was taught that loyalty to the state came before everything else, including one’s family. \
The Helots, whose name means “captives,” were fellow Greeks, originally from Laconia and Messenia, who had been conquered by the Spartans and turned into slaves. The Spartans’ way of life would not have been possible without the Helots, who handled all the day-to-day tasks and unskilled labor required to keep society functioning: They were farmers, domestic servants, nurses and military attendants. \
Spartans, who were outnumbered by the Helots, often treated them brutally and oppressively in an effort to prevent uprisings. Spartans would humiliate the Helots by doing such things as forcing them to get debilitatingly drunk on wine and then make fools of themselves in public. (This practice was also intended to demonstrate to young people how an adult Spartan should never act, as self-control was a prized trait.) \
Methods of mistreatment could be far more extreme: Spartans were allowed to kill Helots for being too smart or too fit, among other reasons."

result = llm_chain.invoke({"question": query})

print(result.rstrip())

Your max_length is set to 750, but your input_length is only 528. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=264)
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Below is an instruction that describes a Request. Write a Response that accurately answers the Request. Review your answer to ensure it is accurate

 Request: summarize the following text: Sparta, also known as Lacedaemon, was an ancient Greek city-state located primarily in a region of southern Greece called Laconia. The population of Sparta consisted of three main groups: the Spartans, or Spartiates, who were full citizens; the Helots, or serfs/slaves; and the Perioeci, who were neither slaves nor citizens. The Perioeci, whose name means “dwellers-around,” worked as craftsmen and traders, and made weapons for the Spartans. Did you know? The word “spartan” means self-restrained, simple, frugal and austere. The word laconic, which means pithy and concise, is derived from the Spartans, who prized brevity of speech. All healthy male Spartan citizens participated in the compulsory state-sponsored education system, the Agoge, which emphasized obedience, endurance, courage and self-control.